In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [2]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2


In [3]:
image_path = '/content/drive/MyDrive/KT_aivle/CNN/mini_4/cool_roof_images.zip'
label_path = "/content/drive/MyDrive/KT_aivle/CNN/mini_4/cool_roof_yolo_labels.zip"
test_path = "/content/drive/MyDrive/KT_aivle/CNN/mini_4/cool_roof_yolo_test.zip"
ex_train_path = '/content/drive/MyDrive/KT_aivle/CNN/mini_4/ex_train.zip' # 211개의 data를 수집, roboflow 증강을 통해 885 개의 train data

In [4]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[-10:-4]}/')

In [5]:
dataset_extract(image_path)

Extracting: 100%|██████████| 200/200 [00:03<00:00, 50.89files/s]


In [6]:
dataset_extract(label_path)

Extracting: 100%|██████████| 204/204 [00:00<00:00, 517.92files/s]


In [7]:
dataset_extract(test_path)

Extracting: 100%|██████████| 26/26 [00:00<00:00, 97.65files/s]


In [8]:
dataset_extract(ex_train_path)

Extracting: 100%|██████████| 1799/1799 [00:00<00:00, 1892.37files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [9]:
!mkdir /content/datasets
!mkdir /content/datasets/train; mkdir /content/datasets/valid;
!mkdir /content/datasets/train/images;mkdir /content/datasets/train/labels;
!mkdir /content/datasets/valid/images;mkdir /content/datasets/valid/labels;

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

# **Load ex_train**

In [10]:
img_path = '/content/images/'
lab_path = '/content/labels/obj_train_data/'
ex_img_path = '/content/_train/train/images/'
ex_img_list = os.listdir(ex_img_path)
ex_lab_path = '/content/_train/train/labels/'
ex_lab_list = os.listdir(ex_lab_path)

for ex_tr_img, ex_tr_lab in zip(ex_img_list,ex_lab_list) :
    shutil.move(ex_img_path + ex_tr_img, img_path)
    shutil.move(ex_lab_path + ex_tr_lab, lab_path)

# Load Img

In [11]:
# Dataset metadata 입력
img_path = '/content/images/'
lab_path = '/content/labels/obj_train_data/'

img_list = os.listdir(img_path)
img_list = sorted(img_list)
lab_list = os.listdir(lab_path)
lab_list = sorted(lab_list)


# 파일 담아오기
data_files = []
for img, lab in zip(img_list,lab_list):
    data_files.append([glob.glob(img_path+img), glob.glob(lab_path+lab)])

# img size (640,640)

In [12]:
# Pillow 라이브러리 불러오기
from PIL import Image

for i in tqdm(range(1,201)):
  img = Image.open(f'/content/images/roof ({i}).jpg')
  img = img.resize((640,640))
  img.save(f'/content/images/roof ({i}).jpg')


#test_img

test_path = "/content/o_test/test/images/"

for test_file in tqdm(os.listdir(test_path)):
  img = Image.open(test_path + test_file)
  img = img.resize((640,640))
  img.save(test_path + test_file)


100%|██████████| 10/10 [00:00<00:00, 168.46it/s]


# HSV

In [13]:
#HSV
import cv2
for i in tqdm(os.listdir(img_path)):
  rgbimg = cv2.imread(img_path + i)
  hsvimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2HSV)
  cv2.imwrite(img_path + i, hsvimg)



for test_file in tqdm(os.listdir(test_path)):
  rgbimg = cv2.imread(test_path + test_file)
  hsvimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2HSV)
  cv2.imwrite(test_path + test_file, hsvimg)


100%|██████████| 10/10 [00:00<00:00, 64.78it/s]


# Gray scale

In [ ]:
#GRAY
import cv2
for i in tqdm(os.listdir(img_path)):
  rgbimg = cv2.imread(img_path + i)
  grayimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2GRAY)
  cv2.imwrite(img_path + i, grayimg)

for test_file in tqdm(os.listdir(test_path)):
  rgbimg = cv2.imread(test_path + test_file)
  grayimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2GRAY)
  cv2.imwrite(test_path + test_file, grayimg)

100%|██████████| 10/10 [00:00<00:00, 34.19it/s]


# Sharping Filter

In [14]:
def sharp(img):
    kernel_sharp = np.array((
        [-2, -2, -2],
        [-2, 17, -2],
        [-2, -2, -2]), dtype='int')
    return cv2.filter2D(img, -1, kernel_sharp)

In [15]:
img_path = "/content/images/"
for i in tqdm(os.listdir(img_path)):
  img = cv2.imread(img_path + i)
  sh_img = sharp(img)
  cv2.imwrite(f'/content/images/{i}.jpg', sh_img)

for test_file in tqdm(os.listdir(test_path)):
  img = cv2.imread(test_path + test_file)
  sh_img = sharp(img)
  cv2.imwrite(test_path + test_file, sh_img)

100%|██████████| 10/10 [00:00<00:00, 16.01it/s]


# data split

In [14]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data_files, test_size=.2, random_state=2024)

In [15]:
train_path = '/content/datasets/train/'
valid_path = '/content/datasets/valid/'

for tr_img, tr_lab in train:
    shutil.move(tr_img[0], train_path +'images')
    shutil.move(tr_lab[0], train_path +'labels')

for va_img, va_lab in val:
    shutil.move(va_img[0], valid_path + 'images')
    shutil.move(va_lab[0], valid_path + 'labels')

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [16]:
import yaml

data = {
    'train': '../train/images',
    'val': '../valid/images',
    'nc': 2,
    'names': ['cool roof', 'generic roof']
}

with open('/content/datasets/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [17]:
!pip install ultralytics==8.0.196


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [18]:
from ultralytics import YOLO, settings

In [19]:
model = YOLO(model='yolov8s.pt', task='detect')

100%|██████████| 6.23M/6.23M [00:00<00:00, 93.4MB/s]


In [20]:
settings['datasets_dir'] = '/content/'
settings.update()

In [26]:
model.train(data='/content/datasets/data.yaml',
            epochs=150,
            patience=7,
            pretrained=False,
            verbose=True,
            seed=2024,
            hsv_s=0.8,
            )

New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/datasets/data.yaml, epochs=150, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=Fals

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7823d8226380>
fitness: 0.7245237820414175
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.69635,     0.70386])
names: {0: 'cool roof', 1: 'generic roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.921809744197914, 'metrics/recall(B)': 0.9051061526432911, 'metrics/mAP50(B)': 0.9443138054257494, 'metrics/mAP50-95(B)': 0.7001026683320474, 'fitness': 0.7245237820414175}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.3021093008155647, 'inference': 5.656835670295399, 'loss': 0.0007877701438517065, 'postprocess': 15.714382795694238}

# **Model.val**

In [ ]:
model.val()
# HSV # 640x640 img

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/valid/labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.06it/s]
                   all         40        304      0.655      0.798      0.761      0.523
             cool roof         40         38      0.511      0.763      0.665      0.474
          generic roof         40        266      0.798      0.833      0.857      0.572
Speed: 0.3ms preprocess, 39.3ms inference, 0.0ms loss, 3.7ms post

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aef6a19bcd0>
fitness: 0.5466403483942581
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.47416,     0.57152])
names: {0: 'cool roof', 1: 'generic roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.6546843809671292, 'metrics/recall(B)': 0.7979583944808025, 'metrics/mAP50(B)': 0.7608140102909569, 'metrics/mAP50-95(B)': 0.5228432748501805, 'fitness': 0.5466403483942581}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 0.2777695655822754, 'inference': 39.28025960922241, 'loss': 0.0014543533325195312, 'postprocess': 3.7055552005767822}

In [ ]:
model.val()
# 640x640 img

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/valid/labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.22s/it]
                   all         40        304       0.78      0.846      0.857       0.58
             cool roof         40         38      0.778      0.828      0.822       0.56
          generic roof         40        266      0.782      0.864      0.892      0.601
Speed: 0.4ms preprocess, 28.3ms inference, 0.0ms loss, 2.1ms post

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f7abc1dce50>
fitness: 0.6079904050670565
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.56014,     0.60054])
names: {0: 'cool roof', 1: 'generic roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.7798380411586657, 'metrics/recall(B)': 0.8458619080091132, 'metrics/mAP50(B)': 0.856835360746834, 'metrics/mAP50-95(B)': 0.5803409655470813, 'fitness': 0.6079904050670565}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 0.39931535720825195, 'inference': 28.336292505264282, 'loss': 0.0016093254089355469, 'postprocess': 2.0606577396392822}

In [ ]:
model.val()
# Gray scale # 640x640 img

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/valid/labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.26it/s]
                   all         40        304       0.64      0.735      0.731      0.482
             cool roof         40         38       0.55      0.605      0.612       0.43
          generic roof         40        266       0.73      0.865       0.85      0.533
Speed: 0.5ms preprocess, 28.0ms inference, 0.0ms loss, 4.4ms post

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e447cf65750>
fitness: 0.5065395049287116
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([     0.4302,     0.53297])
names: {0: 'cool roof', 1: 'generic roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.6402749714255201, 'metrics/recall(B)': 0.7349624060150376, 'metrics/mAP50(B)': 0.7311606275087826, 'metrics/mAP50-95(B)': 0.48158160241981485, 'fitness': 0.5065395049287116}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 0.4761993885040283, 'inference': 27.995294332504272, 'loss': 0.028824806213378906, 'postprocess': 4.4429779052734375}

In [ ]:
model.val()
# add 171 train data #HSV

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/valid/labels.cache... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:09<00:00,  1.87s/it]
                   all         75        705      0.844      0.856        0.9      0.636
             cool roof         75        110      0.858      0.782      0.874      0.635
          generic roof         75        595      0.829      0.931      0.927      0.636
Speed: 3.6ms preprocess, 26.3ms inference, 0.0ms loss, 9.9ms post

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78e7e7d6a7d0>
fitness: 0.6622949580364274
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.63533,     0.63634])
names: {0: 'cool roof', 1: 'generic roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.8436824091029425, 'metrics/recall(B)': 0.856455309396486, 'metrics/mAP50(B)': 0.9004210605365607, 'metrics/mAP50-95(B)': 0.6358365022030792, 'fitness': 0.6622949580364274}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 3.5832532246907554, 'inference': 26.261625289916992, 'loss': 0.0014019012451171875, 'postprocess': 9.855489730834961}

In [ ]:
model.val()
# add 171 train data #Grayscale

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/valid/labels.cache... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:10<00:00,  2.03s/it]
                   all         75        705      0.713       0.86      0.826      0.557
             cool roof         75        110      0.655      0.836      0.763      0.539
          generic roof         75        595       0.77      0.884      0.889      0.575
Speed: 1.8ms preprocess, 25.3ms inference, 0.0ms loss, 8.2ms post

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x781df83399c0>
fitness: 0.5842352588633369
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.53923,     0.57545])
names: {0: 'cool roof', 1: 'generic roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.7126536946061484, 'metrics/recall(B)': 0.8601986249045073, 'metrics/mAP50(B)': 0.8263227439073297, 'metrics/mAP50-95(B)': 0.5573366494140044, 'fitness': 0.5842352588633369}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 1.848929723103841, 'inference': 25.251846313476562, 'loss': 0.04163424173990885, 'postprocess': 8.243481318155924}

In [ ]:
model.val()
# add 171 train data #sharping

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/valid/labels.cache... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  80%|████████  | 4/5 [00:05<00:01,  1.41s/it]WARNING ⚠️ NMS time limit 1.050s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:11<00:00,  2.24s/it]
                   all         75        705      0.838      0.823      0.885      0.636
             cool roof         75        110      0.798      0.754

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x789afdc71b10>
fitness: 0.661073604448521
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.60402,     0.66838])
names: {0: 'cool roof', 1: 'generic roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.837861338577407, 'metrics/recall(B)': 0.8233909433421034, 'metrics/mAP50(B)': 0.8849656638831613, 'metrics/mAP50-95(B)': 0.6361967089557832, 'fitness': 0.661073604448521}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 0.5854892730712891, 'inference': 51.752169926961265, 'loss': 0.0013891855875651042, 'postprocess': 21.46709760030111}

In [28]:

pred = model.predict(test_path,conf=.4,save=True, iou=0.2 )


image 1/10 /content/o_test/test/images/Screenshot-2024-04-11-at-11-37-16-AM_png.rf.166f57a24a9398df49f69dc0a8b7a023.jpg: 640x640 1 cool roof, 3 generic roofs, 15.2ms
image 2/10 /content/o_test/test/images/Screenshot-2024-04-11-at-11-38-28-AM_png.rf.ac6bac50f440f206df9ff74d883e7849.jpg: 640x640 5 cool roofs, 9 generic roofs, 14.9ms
image 3/10 /content/o_test/test/images/Screenshot-2024-04-11-at-11-38-56-AM_png.rf.8649fb416e9c3b26ea9072feefaf3d96.jpg: 640x640 11 cool roofs, 8 generic roofs, 16.3ms
image 4/10 /content/o_test/test/images/Screenshot-2024-04-11-at-11-39-09-AM_png.rf.d98ee834861c44d6a36d289e3813a236.jpg: 640x640 4 cool roofs, 9 generic roofs, 29.3ms
image 5/10 /content/o_test/test/images/Screenshot-2024-04-11-at-11-39-41-AM_png.rf.8112942156eba828d9e0cff676c7a8ab.jpg: 640x640 1 generic roof, 15.1ms
image 6/10 /content/o_test/test/images/Screenshot-2024-04-11-at-11-40-13-AM_png.rf.9a2d2152b81f9afa827ec3851e16dc80.jpg: 640x640 1 cool roof, 3 generic roofs, 20.0ms
image 7/10 /c

In [27]:

pred_path ='/content/runs/detect/predict/'

for test_file in tqdm(os.listdir(pred_path)):
  hsvimg = cv2.imread(pred_path + test_file)
  rgbimg = cv2.cvtColor(hsvimg, cv2.COLOR_HSV2RGB)
  cv2.imwrite('/content/runs/detect/predict/' + test_file, rgbimg)


100%|██████████| 10/10 [00:00<00:00, 45.62it/s]


# Sharping filter를 적용한 이미지의 예측 성승이 우수함.
- generic roof를 판단하는데 건물의 외각선 추출이 성는의 영향을 미침

train data 1085개에 대해서 학습을 진행하고자 하였으나, 시간이 부족하여 성능 비교를 마치지 못하였음.

# 그림자, 무지개 등 과 같은 명도를 표현하는데 RGB보다 유리한 HSV 포멧으로 변경하여 예측

- 주변 건물에 의해서 그림자진 roof의 오분류를 줄이고가 하였으나 성능 개선은 이루어지지 않음

# cool roof 를 분류하는데 도움이 될 것이라는 가정으로 Gray scale 로 변환

- cool roof 를 분류하는데 도움은 있은 것으로 보이나 generic roof 를 구분하는 것에서 성능이 저하된다.